# Gold Layer Exploration and Logistic Regression Modeling

This script:
1. Loads and explores the gold layer feature store
2. Performs EDA on features and target variable
3. Prepares data for modeling (feature selection, train/test split)
4. Trains a logistic regression model
5. Evaluates model performance with metrics and visualizations

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import pyspark
from pyspark.sql.functions import col
import pyspark.sql.functions as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    roc_curve,
    precision_recall_curve,
    average_precision_score
)

In [2]:
# Display all columns
pd.set_option('display.max_columns', None)

In [3]:
# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [4]:
# Initialize Spark
spark = pyspark.sql.SparkSession.builder \
    .appName("explore_gold") \
    .master("local[*]") \
    .config("spark.sql.parquet.mergeSchema", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 11:16:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Configuration
FEATURE_DIR = "/app/datamart/gold/feature_store/"
LABEL_DIR = "/app/datamart/gold/label_store/"
APPLICATION_DIR = "/app/datamart/gold/application_store/"

In [6]:
spark.read.parquet(APPLICATION_DIR).printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- loan_amt: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- application_date: date (nullable = true)



In [7]:
application = spark.read.parquet(APPLICATION_DIR)

In [8]:
application.groupBy("Customer_ID").count().filter("count > 1").show()

[Stage 2:================>                                          (2 + 5) / 7]

+-----------+-----+
|Customer_ID|count|
+-----------+-----+
+-----------+-----+



# LOAD GOLD LAYER DATA

In [9]:
# Get all parquet files
feature_files = spark.read.parquet(FEATURE_DIR)
feature_files.show()

+-----------+--------+------+-------------+----------------+------------------+--------------------+------------------+-------------------------------------------------+------------------------------------------------+-------------------------------------------------+------------------------------------------------+--------------------------------------------------+-------------------------------------------------+-------------------+---------------+--------------+----------------------+--------------------------------+--------------------------+-----------------------------+--------------------------+-------------------------+------------------------------------+------------------------+-----------------------+------------------------+-------------------------+-------------------------+--------------------------+-------------+----------------+----+--------------+--------------+--------------+--------------+-----------+-----------+------------+-----------+-----------+------------+-----

In [10]:
# Get all parquet files
label_files_pattern = LABEL_DIR + "gold_label_store_*.parquet"

label_files = spark.read.parquet(label_files_pattern)
label_files.show()

+--------------------+-----------+-----+----------+-------------+
|             loan_id|Customer_ID|label| label_def|snapshot_date|
+--------------------+-----------+-----+----------+-------------+
|CUS_0x1037_2023_0...| CUS_0x1037|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1069_2023_0...| CUS_0x1069|    0|30dpd_6mob|   2023-07-01|
|CUS_0x114a_2023_0...| CUS_0x114a|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1184_2023_0...| CUS_0x1184|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1297_2023_0...| CUS_0x1297|    1|30dpd_6mob|   2023-07-01|
|CUS_0x12fb_2023_0...| CUS_0x12fb|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1325_2023_0...| CUS_0x1325|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1341_2023_0...| CUS_0x1341|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1375_2023_0...| CUS_0x1375|    1|30dpd_6mob|   2023-07-01|
|CUS_0x13a8_2023_0...| CUS_0x13a8|    0|30dpd_6mob|   2023-07-01|
|CUS_0x13ef_2023_0...| CUS_0x13ef|    0|30dpd_6mob|   2023-07-01|
|CUS_0x1440_2023_0...| CUS_0x1440|    0|30dpd_6mob|   2023-07-01|
|CUS_0x144

In [11]:
# Convert to Pandas for easier analysis and modeling
print("\nConverting to Pandas...")
feature_pd = feature_files.toPandas()
print(f"✓ Pandas DataFrame shape: {feature_pd.shape}")


Converting to Pandas...


✓ Pandas DataFrame shape: (12500, 164)


In [12]:
# Convert to Pandas for easier analysis and modeling
print("\nConverting to Pandas...")
label_pd = label_files.toPandas()
print(f"✓ Pandas DataFrame shape: {label_pd.shape}")


Converting to Pandas...


[Stage 11:=======>                                                  (1 + 7) / 8]

✓ Pandas DataFrame shape: (21474, 5)


# BASIC EXPLORATION

In [13]:
feature_pd.head(10)

,Customer_ID,loan_amt,tenure,Annual_Income,Outstanding_Debt,Num_of_Loan_active,DTI,log_Annual_Income,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Large_value_payments,Credit_Mix_Standard,Credit_Mix_Good,Credit_Mix_Bad,Type_of_Loan_Auto_Loan,Type_of_Loan_Credit_Builder_Loan,Type_of_Loan_Personal_Loan,Type_of_Loan_Home_Equity_Loan,Type_of_Loan_Mortgage_Loan,Type_of_Loan_Student_Loan,Type_of_Loan_Debt_Consolidation_Loan,Type_of_Loan_Payday_Loan,Credit_History_Age_Year,Credit_History_Age_Month,Num_of_Delayed_Payment_3m,Num_of_Delayed_Payment_6m,Num_of_Delayed_Payment_12m,max_dpd_prior,ever_30dpd_prior,Age,age_band_18_24,age_band_25_34,age_band_35_44,age_band_45_54,age_band_55,fe_1_sum_7d,fe_1_mean_7d,fe_1_std_7d,fe_2_sum_7d,fe_2_mean_7d,fe_2_std_7d,fe_3_sum_7d,fe_3_mean_7d,fe_3_std_7d,fe_4_sum_7d,fe_4_mean_7d,fe_4_std_7d,fe_5_sum_7d,fe_5_mean_7d,fe_5_std_7d,fe_6_sum_7d,fe_6_mean_7d,fe_6_std_7d,fe_7_sum_7d,fe_7_mean_7d,fe_7_std_7d,fe_8_sum_7d,fe_8_mean_7d,fe_8_std_7d,fe_9_sum_7d,fe_9_mean_7d,fe_9_std_7d,fe_10_sum_7d,fe_10_mean_7d,fe_10_std_7d,fe_11_sum_7d,fe_11_mean_7d,fe_11_std_7d,fe_12_sum_7d,fe_12_mean_7d,fe_12_std_7d,fe_13_sum_7d,fe_13_mean_7d,fe_13_std_7d,fe_14_sum_7d,fe_14_mean_7d,fe_14_std_7d,fe_15_sum_7d,fe_15_mean_7d,fe_15_std_7d,fe_16_sum_7d,fe_16_mean_7d,fe_16_std_7d,fe_17_sum_7d,fe_17_mean_7d,fe_17_std_7d,fe_18_sum_7d,fe_18_mean_7d,fe_18_std_7d,fe_19_sum_7d,fe_19_mean_7d,fe_19_std_7d,fe_20_sum_7d,fe_20_mean_7d,fe_20_std_7d,fe_1_sum_30d,fe_1_mean_30d,fe_1_std_30d,fe_2_sum_30d,fe_2_mean_30d,fe_2_std_30d,fe_3_sum_30d,fe_3_mean_30d,fe_3_std_30d,fe_4_sum_30d,fe_4_mean_30d,fe_4_std_30d,fe_5_sum_30d,fe_5_mean_30d,fe_5_std_30d,fe_6_sum_30d,fe_6_mean_30d,fe_6_std_30d,fe_7_sum_30d,fe_7_mean_30d,fe_7_std_30d,fe_8_sum_30d,fe_8_mean_30d,fe_8_std_30d,fe_9_sum_30d,fe_9_mean_30d,fe_9_std_30d,fe_10_sum_30d,fe_10_mean_30d,fe_10_std_30d,fe_11_sum_30d,fe_11_mean_30d,fe_11_std_30d,fe_12_sum_30d,fe_12_mean_30d,fe_12_std_30d,fe_13_sum_30d,fe_13_mean_30d,fe_13_std_30d,fe_14_sum_30d,fe_14_mean_30d,fe_14_std_30d,fe_15_sum_30d,fe_15_mean_30d,fe_15_std_30d,fe_16_sum_30d,fe_16_mean_30d,fe_16_std_30d,fe_17_sum_30d,fe_17_mean_30d,fe_17_std_30d,fe_18_sum_30d,fe_18_mean_30d,fe_18_std_30d,fe_19_sum_30d,fe_19_mean_30d,fe_19_std_30d,fe_20_sum_30d,fe_20_mean_30d,fe_20_std_30d,estimated_EMI,EMI_to_income,requested_amount,requested_tenure,snapshot_date,application_date
0,CUS_0x1000,10000,10,30625.940,1562.91,2.0,0.051032,10.329603,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,10,9,26.0,26.0,26.0,26.0,0,18.0,1,0,0,0,0,58,58.0,0.0,15,15.0,0.0,18,18.0,0.0,-101,-101.0,0.0,48,48.0,0.0,-33,-33.0,0.0,238,238.0,0.0,119,119.0,0.0,75,75.0,0.0,52,52.0,0.0,39,39.0,0.0,131,131.0,0.0,-34,-34.0,0.0,145,145.0,0.0,-61,-61.0,0.0,39,39.0,0.0,52,52.0,0.0,80,80.0,0.0,4,4.0,0.0,31,31.0,0.0,150,75.0,24.041631,121,60.5,64.346717,124,62.0,62.225397,-62,-31.0,98.994949,336,168.0,169.705627,159,79.5,159.099026,492,246.0,11.313708,213,106.5,17.677670,245,122.5,67.175144,66,33.0,26.870058,203,101.5,88.388348,100,50.0,114.551299,105,52.5,122.329473,265,132.5,17.677670,8,4.0,91.923882,211,105.5,94.045202,138,69.0,24.041631,155,77.5,3.535534,68,34.0,42.426407,93,46.5,21.920310,1055.820766,0.413697,10000,10,2023-05-01,2023-05-01
1,CUS_0x108a,10000,10,36982.360,4882.12,9.0,0.132012,10.518196,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,7,9,15.0,15.0,15.0,15.0,0,38.0,0,0,1,0,0,77,77.0,0.0,80,80.0,0.0,44,44.0,0.0,-95,-95.0,0.0,161,161.0,0.0,126,126.0,0.0,50,50.0,0.0,-122,-122.0,0.0,105,105.0,0.0,112,112.0,0.0,170,170.0,0.0,232,232.0,0.0,268,268.0,0.0,-50,-50.0,0.0,12,12.0,0.0,123,123.0,0.0,137,137.0,0.0,105,105.0,0.0,31,31.0,0.0,416,416.0,0.0,140,70.0,9.899495,144,72.0,11.313708,275,137.5,132.228968,-16,-8.0,123.036580,344,172.0,15.556349,454,227.0,142.835570,96,48.0,2.828427,82,41.0,230

In [14]:
label_pd.head(10)

,loan_id,Customer_ID,label,label_def,snapshot_date
0,CUS_0x1037_2023_01_01,CUS_0x1037,0,30dpd_6mob,2023-07-01
1,CUS_0x1069_2023_01_01,CUS_0x1069,0,30dpd_6mob,2023-07-01
2,CUS_0x114a_2023_01_01,CUS_0x114a,0,30dpd_6mob,2023-07-01
3,CUS_0x1184_2023_01_01,CUS_0x1184,0,30dpd_6mob,2023-07-01
4,CUS_0x1297_2023_01_01,CUS_0x1297,1,30dpd_6mob,2023-07-01
5,CUS_0x12fb_2023_01_01,CUS_0x12fb,0,30dpd_6mob,2023-07-01
6,CUS_0x1325_2023_01_01,CUS_0x1325,0,30dpd_6mob,2023-07-01
7,CUS_0x1341_2023_01_01,CUS_0x1341,0,30dpd_6mob,2023-07-01
8,CUS_0x1375_2023_01_01,CUS_0x1375,1,30dpd_6mob,2023-07-01
9,CUS_0x13a8_2023_01_01,CUS_0x13a8,0,30dpd_6mob,2023-07-01


In [15]:
label_pd[label_pd['label'] == 1]

,loan_id,Customer_ID,label,label_def,snapshot_date
4,CUS_0x1297_2023_01_01,CUS_0x1297,1,30dpd_6mob,2023-07-01
8,CUS_0x1375_2023_01_01,CUS_0x1375,1,30dpd_6mob,2023-07-01
17,CUS_0x15ad_2023_01_01,CUS_0x15ad,1,30dpd_6mob,2023-07-01
18,CUS_0x1630_2023_01_01,CUS_0x1630,1,30dpd_6mob,2023-07-01
22,CUS_0x1733_2023_01_01,CUS_0x1733,1,30dpd_6mob,2023-07-01
...,...,...,...,...,...
13445,CUS_0xc075_2023_09_01,CUS_0xc075,1,30dpd_6mob,2024-03-01
13448,CUS_0xc122_2023_09_01,CUS_0xc122,1,30dpd_6mob,2024-03-01
13450,CUS_0xc1fe_2023_09_01,CUS_0xc1fe,1,30dpd_6mob,2024-03-01
13454,CUS_0xc2b7_2023_09_01,CUS_0xc2b7,1,30dpd_6mob,2024-03-01


In [16]:
df = spark.read.parquet("/app/datamart/silver/lms_loan_daily")
df.select("snapshot_date").distinct().show()

[Stage 13:==============>                                           (2 + 6) / 8]

+-------------+
|snapshot_date|
+-------------+
|   2025-02-01|
|   2024-08-01|
|   2025-03-01|
|   2024-05-01|
|   2023-08-01|
|   2025-01-01|
|   2025-04-01|
|   2023-10-01|
|   2023-09-01|
|   2024-10-01|
|   2024-04-01|
|   2023-12-01|
|   2024-03-01|
|   2023-07-01|
|   2023-11-01|
|   2025-05-01|
|   2024-12-01|
|   2023-06-01|
|   2024-09-01|
|   2023-05-01|
+-------------+
only showing top 20 rows



In [17]:
df.groupBy("mob").count().orderBy("mob").show()
df.select(F.max("dpd"), F.min("dpd")).show()

+---+-----+
|mob|count|
+---+-----+
|  0|12500|
|  1|12500|
|  2|12500|
|  3|12500|
|  4|12500|
|  5|12500|
|  6|12500|
|  7|12500|
|  8|12500|
|  9|12500|
| 10|12500|
+---+-----+



[Stage 19:=====================>                                    (3 + 5) / 8]

+--------+--------+
|max(dpd)|min(dpd)|
+--------+--------+
|     306|       0|
+--------+--------+



In [18]:
snapshot_date = "2023-07-01"  # pick one that exists
df = df.filter(F.col("snapshot_date") == F.lit(snapshot_date))
df = df.filter(F.col("mob") == 6)
df = df.withColumn("label", F.when(F.col("dpd") >= 30, 1).otherwise(0))
df.groupBy("label").count().show()


[Stage 22:=======>                                                  (1 + 7) / 8]

+-----+-----+
|label|count|
+-----+-----+
|    1|  140|
|    0|  390|
+-----+-----+



In [19]:
# Missing values
print("\n--- Missing Values Summary ---")
missing = feature_pd.isnull().sum()
missing_pct = (missing / len(feature_pd)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct
})
missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_pct', ascending=False)

if len(missing_df) > 0:
    print(f"\nFeatures with missing values: {len(missing_df)}")
    print("\nTop 10 features with most missing values:")
    print(missing_df.head(10))
else:
    print("✓ No missing values found!")


--- Missing Values Summary ---
✓ No missing values found!


In [20]:
# Missing values
print("\n--- Missing Values Summary ---")
missing = label_pd.isnull().sum()
missing_pct = (missing / len(label_pd)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct
})
missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_pct', ascending=False)

if len(missing_df) > 0:
    print(f"\nFeatures with missing values: {len(missing_df)}")
    print("\nTop 10 features with most missing values:")
    print(missing_df.head(10))
else:
    print("✓ No missing values found!")


--- Missing Values Summary ---
✓ No missing values found!


# FEATURE ANALYSIS

In [21]:
# Identify feature types
numeric_features = feature_pd.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = feature_pd.select_dtypes(include=['object', 'category']).columns.tolist()

# Remove non-feature columns
exclude_cols = ['loan_id', 'Customer_ID', 'application_date', 'snapshot_date', 'default_label']
numeric_features = [f for f in numeric_features if f not in exclude_cols]
categorical_features = [f for f in categorical_features if f not in exclude_cols]

print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")


Numeric features: 161
Categorical features: 0


In [22]:
feature_pd.describe()

,loan_amt,tenure,Annual_Income,Outstanding_Debt,Num_of_Loan_active,DTI,log_Annual_Income,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Large_value_payments,Credit_Mix_Standard,Credit_Mix_Good,Credit_Mix_Bad,Type_of_Loan_Auto_Loan,Type_of_Loan_Credit_Builder_Loan,Type_of_Loan_Personal_Loan,Type_of_Loan_Home_Equity_Loan,Type_of_Loan_Mortgage_Loan,Type_of_Loan_Student_Loan,Type_of_Loan_Debt_Consolidation_Loan,Type_of_Loan_Payday_Loan,Credit_History_Age_Year,Credit_History_Age_Month,Num_of_Delayed_Payment_3m,Num_of_Delayed_Payment_6m,Num_of_Delayed_Payment_12m,max_dpd_prior,ever_30dpd_prior,Age,age_band_18_24,age_band_25_34,age_band_35_44,age_band_45_54,age_band_55,fe_1_sum_7d,fe_1_mean_7d,fe_1_std_7d,fe_2_sum_7d,fe_2_mean_7d,fe_2_std_7d,fe_3_sum_7d,fe_3_mean_7d,fe_3_std_7d,fe_4_sum_7d,fe_4_mean_7d,fe_4_std_7d,fe_5_sum_7d,fe_5_mean_7d,fe_5_std_7d,fe_6_sum_7d,fe_6_mean_7d,fe_6_std_7d,fe_7_sum_7d,fe_7_mean_7d,fe_7_std_7d,fe_8_sum_7d,fe_8_mean_7d,fe_8_std_7d,fe_9_sum_7d,fe_9_mean_7d,fe_9_std_7d,fe_10_sum_7d,fe_10_mean_7d,fe_10_std_7d,fe_11_sum_7d,fe_11_mean_7d,fe_11_std_7d,fe_12_sum_7d,fe_12_mean_7d,fe_12_std_7d,fe_13_sum_7d,fe_13_mean_7d,fe_13_std_7d,fe_14_sum_7d,fe_14_mean_7d,fe_14_std_7d,fe_15_sum_7d,fe_15_mean_7d,fe_15_std_7d,fe_16_sum_7d,fe_16_mean_7d,fe_16_std_7d,fe_17_sum_7d,fe_17_mean_7d,fe_17_std_7d,fe_18_sum_7d,fe_18_mean_7d,fe_18_std_7d,fe_19_sum_7d,fe_19_mean_7d,fe_19_std_7d,fe_20_sum_7d,fe_20_mean_7d,fe_20_std_7d,fe_1_sum_30d,fe_1_mean_30d,fe_1_std_30d,fe_2_sum_30d,fe_2_mean_30d,fe_2_std_30d,fe_3_sum_30d,fe_3_mean_30d,fe_3_std_30d,fe_4_sum_30d,fe_4_mean_30d,fe_4_std_30d,fe_5_sum_30d,fe_5_mean_30d,fe_5_std_30d,fe_6_sum_30d,fe_6_mean_30d,fe_6_std_30d,fe_7_sum_30d,fe_7_mean_30d,fe_7_std_30d,fe_8_sum_30d,fe_8_mean_30d,fe_8_std_30d,fe_9_sum_30d,fe_9_mean_30d,fe_9_std_30d,fe_10_sum_30d,fe_10_mean_30d,fe_10_std_30d,fe_11_sum_30d,fe_11_mean_30d,fe_11_std_30d,fe_12_sum_30d,fe_12_mean_30d,fe_12_std_30d,fe_13_sum_30d,fe_13_mean_30d,fe_13_std_30d,fe_14_sum_30d,fe_14_mean_30d,fe_14_std_30d,fe_15_sum_30d,fe_15_mean_30d,fe_15_std_30d,fe_16_sum_30d,fe_16_mean_30d,fe_16_std_30d,fe_17_sum_30d,fe_17_mean_30d,fe_17_std_30d,fe_18_sum_30d,fe_18_mean_30d,fe_18_std_30d,fe_19_sum_30d,fe_19_mean_30d,fe_19_std_30d,fe_20_sum_30d,fe_20_mean_30d,fe_20_std_30d,estimated_EMI,EMI_to_income,requested_amount,requested_tenure
count,12500.0,12500.0,1.250000e+04,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.00000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,1

In [24]:
feature_pd.dtypes

Customer_ID          object
loan_amt              int32
tenure                int32
Annual_Income       float64
Outstanding_Debt    float64
                     ...   
EMI_to_income       float64
requested_amount      int32
requested_tenure      int32
snapshot_date        object
application_date     object
Length: 164, dtype: object

In [25]:
feature_pd.columns.values

array(['Customer_ID', 'loan_amt', 'tenure', 'Annual_Income',
       'Outstanding_Debt', 'Num_of_Loan_active', 'DTI',
       'log_Annual_Income',
       'Payment_Behaviour_High_spent_Small_value_payments',
       'Payment_Behaviour_Low_spent_Large_value_payments',
       'Payment_Behaviour_Low_spent_Medium_value_payments',
       'Payment_Behaviour_Low_spent_Small_value_payments',
       'Payment_Behaviour_High_spent_Medium_value_payments',
       'Payment_Behaviour_High_spent_Large_value_payments',
       'Credit_Mix_Standard', 'Credit_Mix_Good', 'Credit_Mix_Bad',
       'Type_of_Loan_Auto_Loan', 'Type_of_Loan_Credit_Builder_Loan',
       'Type_of_Loan_Personal_Loan', 'Type_of_Loan_Home_Equity_Loan',
       'Type_of_Loan_Mortgage_Loan', 'Type_of_Loan_Student_Loan',
       'Type_of_Loan_Debt_Consolidation_Loan', 'Type_of_Loan_Payday_Loan',
       'Credit_History_Age_Year', 'Credit_History_Age_Month',
       'Num_of_Delayed_Payment_3m', 'Num_of_Delayed_Payment_6m',
       'Num_of_Delayed

In [33]:
# Key numeric features
key_numeric = ['DTI', 'log_Annual_Income', 'Annual_Income', 'Age', 
               'Credit_History_Age_Year', 'Num_of_Loan_active',
               'Num_of_Delayed_Payment_3m', 'Num_of_Delayed_Payment_6m',
               'Num_of_Delayed_Payment_12m', 'max_dpd_prior', 'ever_30dpd_prior', 
               'EMI_to_income','fe_1_sum_7d', 'fe_1_mean_7d',
       'fe_1_std_7d', 'fe_2_sum_7d', 'fe_2_mean_7d', 'fe_2_std_7d',
       'fe_3_sum_7d', 'fe_3_mean_7d', 'fe_3_std_7d', 'fe_4_sum_7d',
       'fe_4_mean_7d', 'fe_4_std_7d', 'fe_5_sum_7d', 'fe_5_mean_7d',
       'fe_5_std_7d', 'fe_6_sum_7d', 'fe_6_mean_7d', 'fe_6_std_7d',
       'fe_7_sum_7d', 'fe_7_mean_7d', 'fe_7_std_7d', 'fe_8_sum_7d',
       'fe_8_mean_7d', 'fe_8_std_7d', 'fe_9_sum_7d', 'fe_9_mean_7d',
       'fe_9_std_7d', 'fe_10_sum_7d', 'fe_10_mean_7d', 'fe_10_std_7d',
       'fe_11_sum_7d', 'fe_11_mean_7d', 'fe_11_std_7d', 'fe_12_sum_7d',
       'fe_12_mean_7d', 'fe_12_std_7d', 'fe_13_sum_7d', 'fe_13_mean_7d',
       'fe_13_std_7d', 'fe_14_sum_7d', 'fe_14_mean_7d', 'fe_14_std_7d',
       'fe_15_sum_7d', 'fe_15_mean_7d', 'fe_15_std_7d', 'fe_16_sum_7d',
       'fe_16_mean_7d', 'fe_16_std_7d', 'fe_17_sum_7d', 'fe_17_mean_7d',
       'fe_17_std_7d', 'fe_18_sum_7d', 'fe_18_mean_7d', 'fe_18_std_7d',
       'fe_19_sum_7d', 'fe_19_mean_7d', 'fe_19_std_7d', 'fe_20_sum_7d',
       'fe_20_mean_7d', 'fe_20_std_7d', 'fe_1_sum_30d', 'fe_1_mean_30d',
       'fe_1_std_30d', 'fe_2_sum_30d', 'fe_2_mean_30d', 'fe_2_std_30d',
       'fe_3_sum_30d', 'fe_3_mean_30d', 'fe_3_std_30d', 'fe_4_sum_30d',
       'fe_4_mean_30d', 'fe_4_std_30d', 'fe_5_sum_30d', 'fe_5_mean_30d',
       'fe_5_std_30d', 'fe_6_sum_30d', 'fe_6_mean_30d', 'fe_6_std_30d',
       'fe_7_sum_30d', 'fe_7_mean_30d', 'fe_7_std_30d', 'fe_8_sum_30d',
       'fe_8_mean_30d', 'fe_8_std_30d', 'fe_9_sum_30d', 'fe_9_mean_30d',
       'fe_9_std_30d', 'fe_10_sum_30d', 'fe_10_mean_30d', 'fe_10_std_30d',
       'fe_11_sum_30d', 'fe_11_mean_30d', 'fe_11_std_30d',
       'fe_12_sum_30d', 'fe_12_mean_30d', 'fe_12_std_30d',
       'fe_13_sum_30d', 'fe_13_mean_30d', 'fe_13_std_30d',
       'fe_14_sum_30d', 'fe_14_mean_30d', 'fe_14_std_30d',
       'fe_15_sum_30d', 'fe_15_mean_30d', 'fe_15_std_30d',
       'fe_16_sum_30d', 'fe_16_mean_30d', 'fe_16_std_30d',
       'fe_17_sum_30d', 'fe_17_mean_30d', 'fe_17_std_30d',
       'fe_18_sum_30d', 'fe_18_mean_30d', 'fe_18_std_30d',
       'fe_19_sum_30d', 'fe_19_mean_30d', 'fe_19_std_30d',
       'fe_20_sum_30d', 'fe_20_mean_30d', 'fe_20_std_30d',
       'estimated_EMI']
key_numeric = [f for f in key_numeric if f in feature_pd.columns]

print("\n--- Key Numeric Features Statistics ---")
feature_pd[key_numeric].describe()


--- Key Numeric Features Statistics ---


,DTI,log_Annual_Income,Annual_Income,Age,Credit_History_Age_Year,Num_of_Loan_active,Num_of_Delayed_Payment_3m,Num_of_Delayed_Payment_6m,Num_of_Delayed_Payment_12m,max_dpd_prior,ever_30dpd_prior,EMI_to_income,fe_1_sum_7d,fe_1_mean_7d,fe_1_std_7d,fe_2_sum_7d,fe_2_mean_7d,fe_2_std_7d,fe_3_sum_7d,fe_3_mean_7d,fe_3_std_7d,fe_4_sum_7d,fe_4_mean_7d,fe_4_std_7d,fe_5_sum_7d,fe_5_mean_7d,fe_5_std_7d,fe_6_sum_7d,fe_6_mean_7d,fe_6_std_7d,fe_7_sum_7d,fe_7_mean_7d,fe_7_std_7d,fe_8_sum_7d,fe_8_mean_7d,fe_8_std_7d,fe_9_sum_7d,fe_9_mean_7d,fe_9_std_7d,fe_10_sum_7d,fe_10_mean_7d,fe_10_std_7d,fe_11_sum_7d,fe_11_mean_7d,fe_11_std_7d,fe_12_sum_7d,fe_12_mean_7d,fe_12_std_7d,fe_13_sum_7d,fe_13_mean_7d,fe_13_std_7d,fe_14_sum_7d,fe_14_mean_7d,fe_14_std_7d,fe_15_sum_7d,fe_15_mean_7d,fe_15_std_7d,fe_16_sum_7d,fe_16_mean_7d,fe_16_std_7d,fe_17_sum_7d,fe_17_mean_7d,fe_17_std_7d,fe_18_sum_7d,fe_18_mean_7d,fe_18_std_7d,fe_19_sum_7d,fe_19_mean_7d,fe_19_std_7d,fe_20_sum_7d,fe_20_mean_7d,fe_20_std_7d,fe_1_sum_30d,fe_1_mean_30d,fe_1_std_30d,fe_2_sum_30d,fe_2_mean_30d,fe_2_std_30d,fe_3_sum_30d,fe_3_mean_30d,fe_3_std_30d,fe_4_sum_30d,fe_4_mean_30d,fe_4_std_30d,fe_5_sum_30d,fe_5_mean_30d,fe_5_std_30d,fe_6_sum_30d,fe_6_mean_30d,fe_6_std_30d,fe_7_sum_30d,fe_7_mean_30d,fe_7_std_30d,fe_8_sum_30d,fe_8_mean_30d,fe_8_std_30d,fe_9_sum_30d,fe_9_mean_30d,fe_9_std_30d,fe_10_sum_30d,fe_10_mean_30d,fe_10_std_30d,fe_11_sum_30d,fe_11_mean_30d,fe_11_std_30d,fe_12_sum_30d,fe_12_mean_30d,fe_12_std_30d,fe_13_sum_30d,fe_13_mean_30d,fe_13_std_30d,fe_14_sum_30d,fe_14_mean_30d,fe_14_std_30d,fe_15_sum_30d,fe_15_mean_30d,fe_15_std_30d,fe_16_sum_30d,fe_16_mean_30d,fe_16_std_30d,fe_17_sum_30d,fe_17_mean_30d,fe_17_std_30d,fe_18_sum_30d,fe_18_mean_30d,fe_18_std_30d,fe_19_sum_30d,fe_19_mean_30d,fe_19_std_30d,fe_20_sum_30d,fe_20_mean_30d,fe_20_std_30d,estimated_EMI
count,12500.000000,12500.000000,1.250000e+04,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.0,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.00000,12500.000000,12500.000000,12500.00000,12500.000000,12500.00000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,12500.000000,1.250000e+04
mean,0.060092,10.587421,1.616206e+05,31.918480,18.235920,3.099440,33.024960,33.024960,33.024960,33.024960,0.008400,0.451760,73.987360,73.987360,0.0,74.108400,74.108400,0.0,74.544320,74.544320,0.0,75.621600,75.621600,0.0,76.848720,76.848720,0.0,72.023680,72.023680,0.0,78.411120,78.411120,0.0,79.076400,79.076400,0.0,81.415920,81.415920,0.0,84.483120,84.483120,0.0,72.054400,72.054400,0.0,71.651200,71.651200,0.0,73.057520,73.057520,0.0,70.802480,70.802480,0.0,71.652240,71.652240,0.0,71.310160,71.310160,0.0,72.237280,72.237280,0.0,73.86392

In [34]:
label_pd.head()

,loan_id,Customer_ID,label,label_def,snapshot_date
0,CUS_0x1037_2023_01_01,CUS_0x1037,0,30dpd_6mob,2023-07-01
1,CUS_0x1069_2023_01_01,CUS_0x1069,0,30dpd_6mob,2023-07-01
2,CUS_0x114a_2023_01_01,CUS_0x114a,0,30dpd_6mob,2023-07-01
3,CUS_0x1184_2023_01_01,CUS_0x1184,0,30dpd_6mob,2023-07-01
4,CUS_0x1297_2023_01_01,CUS_0x1297,1,30dpd_6mob,2023-07-01


In [30]:
df = pd.merge(
    feature_pd,
    label_pd,
    left_on=["Customer_ID", "snapshot_date"],
    right_on=["Customer_ID", "snapshot_date"],
    how="inner"
)
df.head()

,Customer_ID,loan_amt,tenure,Annual_Income,Outstanding_Debt,Num_of_Loan_active,DTI,log_Annual_Income,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Large_value_payments,Credit_Mix_Standard,Credit_Mix_Good,Credit_Mix_Bad,Type_of_Loan_Auto_Loan,Type_of_Loan_Credit_Builder_Loan,Type_of_Loan_Personal_Loan,Type_of_Loan_Home_Equity_Loan,Type_of_Loan_Mortgage_Loan,Type_of_Loan_Student_Loan,Type_of_Loan_Debt_Consolidation_Loan,Type_of_Loan_Payday_Loan,Credit_History_Age_Year,Credit_History_Age_Month,Num_of_Delayed_Payment_3m,Num_of_Delayed_Payment_6m,Num_of_Delayed_Payment_12m,max_dpd_prior,ever_30dpd_prior,Age,age_band_18_24,age_band_25_34,age_band_35_44,age_band_45_54,age_band_55,fe_1_sum_7d,fe_1_mean_7d,fe_1_std_7d,fe_2_sum_7d,fe_2_mean_7d,fe_2_std_7d,fe_3_sum_7d,fe_3_mean_7d,fe_3_std_7d,fe_4_sum_7d,fe_4_mean_7d,fe_4_std_7d,fe_5_sum_7d,fe_5_mean_7d,fe_5_std_7d,fe_6_sum_7d,fe_6_mean_7d,fe_6_std_7d,fe_7_sum_7d,fe_7_mean_7d,fe_7_std_7d,fe_8_sum_7d,fe_8_mean_7d,fe_8_std_7d,fe_9_sum_7d,fe_9_mean_7d,fe_9_std_7d,fe_10_sum_7d,fe_10_mean_7d,fe_10_std_7d,fe_11_sum_7d,fe_11_mean_7d,fe_11_std_7d,fe_12_sum_7d,fe_12_mean_7d,fe_12_std_7d,fe_13_sum_7d,fe_13_mean_7d,fe_13_std_7d,fe_14_sum_7d,fe_14_mean_7d,fe_14_std_7d,fe_15_sum_7d,fe_15_mean_7d,fe_15_std_7d,fe_16_sum_7d,fe_16_mean_7d,fe_16_std_7d,fe_17_sum_7d,fe_17_mean_7d,fe_17_std_7d,fe_18_sum_7d,fe_18_mean_7d,fe_18_std_7d,fe_19_sum_7d,fe_19_mean_7d,fe_19_std_7d,fe_20_sum_7d,fe_20_mean_7d,fe_20_std_7d,fe_1_sum_30d,fe_1_mean_30d,fe_1_std_30d,fe_2_sum_30d,fe_2_mean_30d,fe_2_std_30d,fe_3_sum_30d,fe_3_mean_30d,fe_3_std_30d,fe_4_sum_30d,fe_4_mean_30d,fe_4_std_30d,fe_5_sum_30d,fe_5_mean_30d,fe_5_std_30d,fe_6_sum_30d,fe_6_mean_30d,fe_6_std_30d,fe_7_sum_30d,fe_7_mean_30d,fe_7_std_30d,fe_8_sum_30d,fe_8_mean_30d,fe_8_std_30d,fe_9_sum_30d,fe_9_mean_30d,fe_9_std_30d,fe_10_sum_30d,fe_10_mean_30d,fe_10_std_30d,fe_11_sum_30d,fe_11_mean_30d,fe_11_std_30d,fe_12_sum_30d,fe_12_mean_30d,fe_12_std_30d,fe_13_sum_30d,fe_13_mean_30d,fe_13_std_30d,fe_14_sum_30d,fe_14_mean_30d,fe_14_std_30d,fe_15_sum_30d,fe_15_mean_30d,fe_15_std_30d,fe_16_sum_30d,fe_16_mean_30d,fe_16_std_30d,fe_17_sum_30d,fe_17_mean_30d,fe_17_std_30d,fe_18_sum_30d,fe_18_mean_30d,fe_18_std_30d,fe_19_sum_30d,fe_19_mean_30d,fe_19_std_30d,fe_20_sum_30d,fe_20_mean_30d,fe_20_std_30d,estimated_EMI,EMI_to_income,requested_amount,requested_tenure,snapshot_date,application_date,loan_id,label,label_def
0,CUS_0x1000,10000,10,30625.940,1562.91,2.0,0.051032,10.329603,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,10,9,26.0,26.0,26.0,26.0,0,18.0,1,0,0,0,0,58,58.0,0.0,15,15.0,0.0,18,18.0,0.0,-101,-101.0,0.0,48,48.0,0.0,-33,-33.0,0.0,238,238.0,0.0,119,119.0,0.0,75,75.0,0.0,52,52.0,0.0,39,39.0,0.0,131,131.0,0.0,-34,-34.0,0.0,145,145.0,0.0,-61,-61.0,0.0,39,39.0,0.0,52,52.0,0.0,80,80.0,0.0,4,4.0,0.0,31,31.0,0.0,150,75.0,24.041631,121,60.5,64.346717,124,62.0,62.225397,-62,-31.0,98.994949,336,168.0,169.705627,159,79.5,159.099026,492,246.0,11.313708,213,106.5,17.677670,245,122.5,67.175144,66,33.0,26.870058,203,101.5,88.388348,100,50.0,114.551299,105,52.5,122.329473,265,132.5,17.677670,8,4.0,91.923882,211,105.5,94.045202,138,69.0,24.041631,155,77.5,3.535534,68,34.0,42.426407,93,46.5,21.920310,1055.820766,0.413697,10000,10,2023-05-01,2023-05-01,CUS_0x1000_2023_05_01,0,30dpd_0mob
1,CUS_0x108a,10000,10,36982.360,4882.12,9.0,0.132012,10.518196,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,1,1,7,9,15.0,15.0,15.0,15.0,0,38.0,0,0,1,0,0,77,77.0,0.0,80,80.0,0.0,44,44.0,0.0,-95,-95.0,0.0,161,161.0,0.0,126,126.0,0.0,50,50.0,0.0,-122,-122.0,0.0,105,105.0,0.0,112,112.0,0.0,170,170.0,0.0,232,232.0,0.0,268,268.0,0.0,-50,-50.0,0.0,12,12.0,0.0,123,123.0,0.0,137,137.0,0.0,105,105.0,0.0,31,31.0,0.0,416,416.0,0.0,140,70.0,9.899495,144,72.0,11.313708,275,137.5,132.228968,-16,-8.0,123.036580,344,172.0,

In [32]:
# Correlation with target
print("\n--- Top 15 Features Correlated with Default ---")
correlations = []
for col_name in key_numeric:
    if col_name in df.columns:
        try:
            corr = df[['default_label', col_name]].corr().iloc[0, 1]
            if not np.isnan(corr):
                correlations.append((col_name, corr))
        except:
            pass

correlations.sort(key=lambda x: abs(x[1]), reverse=True)

print("\nFeature                              Correlation")
print("-" * 60)
for feat, corr in correlations[:15]:
    direction = "↑ Risk+" if corr > 0 else "↓ Risk-"
    print(f"{feat:35s} {corr:+.4f}  {direction}")


--- Top 15 Features Correlated with Default ---

Feature                              Correlation
------------------------------------------------------------


In [ ]:
df[key_numeric].nunique().sort_values()

In [ ]:
df[key_numeric].dtypes

In [ ]:
# Visualization: Target distribution
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
df['default_label'].value_counts().plot(kind='bar', color=['green', 'red'], alpha=0.7, edgecolor='black')
plt.xlabel('Default Label')
plt.ylabel('Count')
plt.title('Target Variable Distribution')
plt.xticks([0, 1], ['Non-Default (0)', 'Default (1)'], rotation=0)
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
if 'income_band' in df.columns:
    default_by_income = df.groupby('income_band')['default_label'].mean() * 100
    default_by_income.plot(kind='bar', color='coral', alpha=0.7, edgecolor='black')
    plt.xlabel('Income Band')
    plt.ylabel('Default Rate (%)')
    plt.title('Default Rate by Income Band')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
if 'age_band' in df.columns:
    default_by_age = df.groupby('age_band')['default_label'].mean() * 100
    default_by_age.plot(kind='bar', color='skyblue', alpha=0.7, edgecolor='black')
    plt.xlabel('Age Band')
    plt.ylabel('Default Rate (%)')
    plt.title('Default Rate by Age Band')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('gold_target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved: gold_target_distribution.png")

In [ ]:
# Visualization: Feature distributions by default status
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

plot_features = ['DTI', 'Annual_Income', 'Age', 'Num_of_Delayed_Payment_12m', 
                'Credit_History_Age_Year', 'EMI_to_income']
plot_features = [f for f in plot_features if f in df.columns]

for idx, feature in enumerate(plot_features[:6]):
    ax = axes[idx]
    
    non_default = df[df['default_label'] == 0][feature].dropna()
    default = df[df['default_label'] == 1][feature].dropna()
    
    # Handle outliers for better visualization
    if feature in ['DTI', 'EMI_to_income']:
        non_default = non_default.clip(upper=non_default.quantile(0.95))
        default = default.clip(upper=default.quantile(0.95))
    
    ax.hist(non_default, bins=50, alpha=0.5, label='Non-Default', 
           color='green', edgecolor='black', density=True)
    ax.hist(default, bins=50, alpha=0.5, label='Default', 
           color='red', edgecolor='black', density=True)
    
    ax.set_xlabel(feature)
    ax.set_ylabel('Density')
    ax.set_title(f'{feature} Distribution by Default Status')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('gold_feature_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: gold_feature_distributions.png")


# DATA PREPARATION FOR MODELING

In [ ]:
# Select features for modeling
print("\n--- Feature Selection ---")

# Strategy: Use numeric features with low missing values
# Filter out features with >50% missing values
valid_features = []
for feat in numeric_features:
    missing_pct = df[feat].isnull().sum() / len(df) * 100
    if missing_pct < 50:
        valid_features.append(feat)

print(f"Total numeric features: {len(numeric_features)}")
print(f"Valid features (< 50% missing): {len(valid_features)}")

In [ ]:
# Create modeling dataset
print("\n--- Creating Modeling Dataset ---")

# Select features and target
X = df[valid_features].copy()
y = df['default_label'].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

In [ ]:
# Handle missing values (simple imputation with median)
print("\n--- Handling Missing Values ---")
missing_before = X.isnull().sum().sum()
X = X.fillna(X.median())
missing_after = X.isnull().sum().sum()
print(f"Missing values before imputation: {missing_before:,}")
print(f"Missing values after imputation: {missing_after:,}")

In [ ]:
# Handle infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

In [ ]:
# Remove constant features (zero variance)
print("\n--- Removing Constant Features ---")
constant_features = [col for col in X.columns if X[col].nunique() <= 1]
if constant_features:
    print(f"Removing {len(constant_features)} constant features")
    X = X.drop(columns=constant_features)
    print(f"Features after removal: {len(X.columns)}")

In [ ]:
# Feature importance using correlation (for feature selection)
print("\n--- Feature Selection by Correlation ---")
correlations_with_target = []
for col_name in X.columns:
    corr = np.corrcoef(X[col_name], y)[0, 1]
    if not np.isnan(corr):
        correlations_with_target.append((col_name, abs(corr)))

correlations_with_target.sort(key=lambda x: x[1], reverse=True)

# Select top 20 most correlated features
top_n_features = 20
selected_features = [feat for feat, corr in correlations_with_target[:top_n_features]]

print(f"\nSelecting top {top_n_features} features by correlation:")
for i, (feat, corr) in enumerate(correlations_with_target[:top_n_features], 1):
    print(f"  {i:2d}. {feat:40s} {corr:.4f}")

X_selected = X[selected_features].copy()

print(f"\n✓ Final feature matrix shape: {X_selected.shape}")

In [ ]:
# Train-test split
print("\n--- Train-Test Split ---")
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")
print(f"Training default rate: {y_train.mean()*100:.2f}%")
print(f"Test default rate: {y_test.mean()*100:.2f}%")


In [ ]:
# Feature scaling
print("\n--- Feature Scaling (Standardization) ---")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features standardized (mean=0, std=1)")

# LOGISTIC REGRESSION MODEL

In [ ]:
# Calculate class weights for imbalanced data
class_weight_ratio = (len(y_train) - y_train.sum()) / y_train.sum()
print(f"\nClass imbalance ratio: {class_weight_ratio:.2f}:1 (non-default:default)")

In [ ]:
# Train logistic regression
print("\nTraining Logistic Regression with class weights...")

lr_model = LogisticRegression(
    class_weight='balanced',  # Handle class imbalance
    max_iter=1000,
    random_state=42,
    solver='lbfgs'
)

lr_model.fit(X_train_scaled, y_train)

print("✓ Model trained successfully")

In [ ]:
# Feature coefficients
print("\n--- Feature Coefficients (Top 10) ---")
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'coefficient': lr_model.coef_[0]
})
feature_importance['abs_coefficient'] = feature_importance['coefficient'].abs()
feature_importance = feature_importance.sort_values('abs_coefficient', ascending=False)

print("\nFeature                              Coefficient   Impact")
print("-" * 70)
for idx, row in feature_importance.head(10).iterrows():
    impact = "↑ Increases Risk" if row['coefficient'] > 0 else "↓ Decreases Risk"
    print(f"{row['feature']:35s} {row['coefficient']:+8.4f}   {impact}")

# MODEL EVALUATION

In [ ]:
# Predictions
y_train_pred = lr_model.predict(X_train_scaled)
y_test_pred = lr_model.predict(X_test_scaled)

y_train_proba = lr_model.predict_proba(X_train_scaled)[:, 1]
y_test_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

In [ ]:
# Training metrics
print("\n--- Training Set Performance ---")
print(classification_report(y_train, y_train_pred, 
                          target_names=['Non-Default', 'Default']))

train_auc = roc_auc_score(y_train, y_train_proba)
print(f"ROC-AUC Score: {train_auc:.4f}")

# Test metrics
print("\n--- Test Set Performance ---")
print(classification_report(y_test, y_test_pred, 
                          target_names=['Non-Default', 'Default']))

test_auc = roc_auc_score(y_test, y_test_proba)
print(f"ROC-AUC Score: {test_auc:.4f}")

# Check for overfitting
auc_diff = train_auc - test_auc
if auc_diff > 0.05:
    print(f"\n⚠️  WARNING: Possible overfitting detected (AUC diff: {auc_diff:.4f})")
else:
    print(f"\n✓ Model generalizes well (AUC diff: {auc_diff:.4f})")

# Confusion matrix
print("\n--- Confusion Matrix (Test Set) ---")
cm = confusion_matrix(y_test, y_test_pred)
print("\n                 Predicted")
print("               Non-Default  Default")
print(f"Actual Non-Default  {cm[0,0]:6d}     {cm[0,1]:6d}")
print(f"       Default      {cm[1,0]:6d}     {cm[1,1]:6d}")

# VISUALIZATIONS

In [ ]:
# Create comprehensive visualization
fig = plt.figure(figsize=(18, 12))

# 1. Confusion Matrix
ax1 = plt.subplot(2, 3, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Non-Default', 'Default'],
            yticklabels=['Non-Default', 'Default'])
plt.title('Confusion Matrix (Test Set)')
plt.ylabel('Actual')
plt.xlabel('Predicted')

# 2. ROC Curve
ax2 = plt.subplot(2, 3, 2)
fpr, tpr, _ = roc_curve(y_test, y_test_proba)
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {test_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. Precision-Recall Curve
ax3 = plt.subplot(2, 3, 3)
precision, recall, _ = precision_recall_curve(y_test, y_test_proba)
avg_precision = average_precision_score(y_test, y_test_proba)
plt.plot(recall, precision, linewidth=2, label=f'PR Curve (AP = {avg_precision:.4f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True, alpha=0.3)

# 4. Feature Coefficients
ax4 = plt.subplot(2, 3, 4)
top_features = feature_importance.head(15).sort_values('coefficient')
colors = ['red' if x > 0 else 'green' for x in top_features['coefficient']]
plt.barh(range(len(top_features)), top_features['coefficient'], color=colors, alpha=0.7, edgecolor='black')
plt.yticks(range(len(top_features)), top_features['feature'], fontsize=8)
plt.xlabel('Coefficient')
plt.title('Top 15 Feature Coefficients')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(True, alpha=0.3)

# 5. Predicted Probability Distribution
ax5 = plt.subplot(2, 3, 5)
plt.hist(y_test_proba[y_test == 0], bins=50, alpha=0.5, label='Non-Default', 
         color='green', edgecolor='black', density=True)
plt.hist(y_test_proba[y_test == 1], bins=50, alpha=0.5, label='Default', 
         color='red', edgecolor='black', density=True)
plt.xlabel('Predicted Probability')
plt.ylabel('Density')
plt.title('Predicted Probability Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# 6. Calibration plot (optional)
ax6 = plt.subplot(2, 3, 6)
from sklearn.calibration import calibration_curve
fraction_of_positives, mean_predicted_value = calibration_curve(
    y_test, y_test_proba, n_bins=10
)
plt.plot(mean_predicted_value, fraction_of_positives, marker='o', linewidth=2, label='Model')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Perfect Calibration')
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Plot')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('logistic_regression_evaluation.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✓ Saved: logistic_regression_evaluation.png")

# MODEL SUMMARY

In [ ]:
print(f"""
📊 DATASET:
   • Total applications: {len(df):,}
   • Training samples: {len(X_train):,}
   • Test samples: {len(X_test):,}
   • Features used: {len(selected_features)}
   • Default rate: {df['default_label'].mean()*100:.2f}%

🎯 MODEL PERFORMANCE (Test Set):
   • ROC-AUC: {test_auc:.4f}
   • Precision (Default class): {classification_report(y_test, y_test_pred, output_dict=True)['1']['precision']:.4f}
   • Recall (Default class): {classification_report(y_test, y_test_pred, output_dict=True)['1']['recall']:.4f}
   • F1-Score (Default class): {classification_report(y_test, y_test_pred, output_dict=True)['1']['f1-score']:.4f}

🔝 TOP 5 RISK FACTORS (Positive Coefficients):
""")

for idx, row in feature_importance[feature_importance['coefficient'] > 0].head(5).iterrows():
    print(f"   {row['feature']:35s} +{row['coefficient']:.4f}")

print(f"""
🛡️  TOP 5 PROTECTIVE FACTORS (Negative Coefficients):
""")

for idx, row in feature_importance[feature_importance['coefficient'] < 0].head(5).iterrows():
    print(f"   {row['feature']:35s} {row['coefficient']:.4f}")

print("\n" + "="*80)
print("✅ EXPLORATION AND MODELING COMPLETED")
print("="*80)

# 💡 Next Steps:
   1. Try more advanced models (Random Forest, XGBoost, LightGBM)
   2. Perform hyperparameter tuning
   3. Implement cross-validation
   4. Feature engineering (polynomial features, interactions)
   5. Ensemble methods
   6. Deploy model to production

In [ ]:
# Cleanup
spark.stop()
print("\n✓ Spark session stopped")